## Day 6: Guard Gallivant

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
```

The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

* If there is something directly in front of you, turn right 90 degrees.
* Otherwise, take a step forward.

Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```

This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:

```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```

In this example, the guard will visit `41` distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

### Part 1 Plan

An object will represent a guard that tracks their position, direction, and which positions the guard has visited.

In [1]:
const input = true ?
	await Deno.readTextFile("./day6input.txt") :
	`
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...`.trim()
const rows = input.split("\n")

In [2]:
// The directions are in order of turn direction.
enum Direction {
	North = [-1, 0],
	East = [0, 1],
	South = [1, 0],
	West = [0, -1],
}
type Position = [int, int]
type Guard = {
	direction: Direction,
	position: Position,
	visited: string[], // copy of the map
	turn: () => void
}
function moveDirection(p:Position, d:Direction): Position {
	let [pr, pc] = p
	let [vr, vc] = d
	return [pr+vr, pc+vc]
}
// Find the initial position of the guard. Assume the direction is north.
let initialPosition:Position = rows.reduce(function(init:Position|null, row:string, rowi:int): Position {
	const coli = row.indexOf("^")
	if (coli > -1) {
		return [rowi, coli]
	}
	return init
}, null)

const guard:Guard = {
	direction: Direction.North,
	position: initialPosition,
	visited: [...rows],
	markVisited: () => {
		const row = guard.visited[guard.position[0]].split('')
		row[guard.position[1]] = 'X'
		guard.visited[guard.position[0]] = row.join('')
	},
	turn: () => {
		const directions:Direction[] = Object.values(Direction)
		// Adding two to the  index, because Object.values gives the Position and then the enum label
		guard.direction = directions[(directions.indexOf(guard.direction) + 2) % directions.length]
	}
}
while (true) {
	guard.markVisited()
	const [nrow, ncol]:Position = moveDirection(guard.position, guard.direction)
	if (rows[nrow]?.charAt(ncol) == '#') {
		guard.turn()
		continue
	} else if (['.', '^', 'X'].includes(rows[nrow]?.charAt(ncol))) {
		guard.position = [nrow, ncol]
		continue
	}
	break
}
const visited = guard.visited.reduce((sum:int, row:string) => {
	return sum + (row.match(/X/g) || '').length
}, 0)
console.log(`Distinct positions visited: ${visited}`)

Distinct positions visited: 4967
